# Problem 3 Naive Bayes

## 3.1 implement a Naive Bayes classifier

In [1]:
import sys
import math
import itertools
from collections import Counter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import time
from copy import deepcopy

In [2]:
class Model:
    def __init__(self, wordlist):
        self.wordlist = wordlist

    def count_labels(self, data):
        """
        Count the number of positive labels and negative labels.
        Returns (a tuple or a numpy array of two elements):
            * negative_count: a non-negative integer, which represents the number of negative labels (non-spam emails);
            * positive_count: a non-negative integer, which represents the number of positive labels (spam emails).
        """
        # TODO
        result = np.zeros((2))
        pos = 0
        neg = 0
        for i in range(len(data)):
            if data[i][0] == 1:
                pos += 1
            else:
                neg += 1
                
        result[0] = neg
        result[1] = pos
        print("number of spam email ",result[1])
        return result
    
    def count_words(self, wordlist, data):
        """
        Count the number of times that each word appears in emails under a given label.
        Returns (a numpy array):
            * word_counts: a numpy array with shape (2, L), where L is the length of $wordlist,
                - word_counts[0, i] represents the number of times that word $wordlist[i] appears in non-spam (negative) emails, and
                - word_counts[1, i] represents the number of times that word $wordlist[i] appears in spam (positive) emails.
        """
        # TODO
        L = len(wordlist)
        word_counts = np.zeros((2,L))
#         for i in range(L):
#             for j in data:
#                 if j[0] == 0:
#                     #non spam
#                     for word in j[1]:
#                         if word == wordlist[i]:
#                             word_counts[0][i] += 1
#                             break
#                 else:
#                     #spam
#                     for word in j[1]:
#                         if word == wordlist[i]:
#                             word_counts[1][i] += 1
#                             break
        #transform data into ndarray
        data = np.asarray(data)
        print("shape of data: ",data.shape)
        spam_data = data[np.where(data[:,0]==1)][:,1]
        nons_data = data[np.where(data[:,0]==0)][:,1]
        
        
#         print("shape of spam data: ",len(spam_data))
        
        for j in range(L):
            for i in range(len(spam_data)):
#                 row = np.array(spam_data[i])
#                 values,counts = np.unique(row,return_counts=True)
#                 row_dict = dict(zip(values,counts))
                row_set = set(spam_data[i])
                if wordlist[j] in row_set:
                    word_counts[1][j]+=1
        
        for j in range(L):
            for i in range(len(nons_data)):
#                 row = np.array(nons_data[i])
#                 values,counts = np.unique(row,return_counts=True)
#                 row_dict = dict(zip(values,counts))
                row_set = set(nons_data[i])
                if wordlist[j] in row_set:
                    word_counts[0][j]+=1
        
        return word_counts
                    
#         values, counts = np.unique(spam_data, return_counts=True)
#         spam_dict = dict(zip(values,counts))
        
#         values, counts = np.unique(nons_data, return_counts=True)
#         nons_dict = dict(zip(values,counts))
        
#         for i in range(L):
#             word = wordlist[i]
#             if word in spam_dict:
#                 word_counts[1,i] = spam_dict[word]
#                 print(word,spam_dict[word])
#             else:
#                 word_counts[1,i] = 0
            
#             if word in nons_dict:
#                 word_counts[0,i] = nons_dict[word]
#                 print(word,nons_dict[word])
#             else:
#                 word_counts[0,i] = 0
        
# #         print("word_counts: ",word_counts)
#         return word_counts

    def calculate_probability(self, label_counts, word_counts):
        """
        Calculate the probabilities, both the prior and likelihood.
        Returns (a pair of numpy array):
            * prior_probs: a numpy array with shape (2, ), only two elements, where
                - prior_probs[0] is the prior probability of negative labels, and
                - prior_probs[1] is the prior probability of positive labels.
            * likelihood_probs: a numpy array with shape (2, L), where L is the length of the word list,
                - likelihood_probs[0, i] represents the likelihood probability of the $i-th word in the word list, given that the email is non-spam (negative), and
                - likelihood_probs[1, i] represents the likelihood probability of the $i-th word in the word list, given that the email is spam (positive).
        """
        # TODO
        L = word_counts.shape[1]
        prior_probs = np.zeros((2))
        likelihood_probs = np.zeros((2,L))
        prior_probs[0] = label_counts[0] / (label_counts[0]+ label_counts[1])
        prior_probs[1] = label_counts[1] / (label_counts[0]+ label_counts[1])                       
        
        for i in range(word_counts.shape[1]):
            likelihood_probs[0,i] = (word_counts[0,i]+1)/(label_counts[0]+2)
            
            likelihood_probs[1,i] = (word_counts[1,i]+1)/(label_counts[1]+2)
#             print("index ",i," this word count in non-spam + 1",word_counts[0,i]+1,"non-spam total: ",label_counts[0]+2)
#             print("index ",i,"this word count in spam + 1",word_counts[1,i]+1,"spam total: ",label_counts[1]+2)
#             print("index ",i,"likelihood_probs for index i",likelihood_probs[0,i],likelihood_probs[1,i])
            
        return prior_probs,likelihood_probs
        
    def fit(self, data):
        label_counts = self.count_labels(data)
        word_counts = self.count_words(self.wordlist, data)
        
        self.label_counts = label_counts
        self.word_counts = word_counts
        self.prior_probs, self.likelihood_probs = self.calculate_probability(label_counts, word_counts)

        # TO AVOID NUMBER OVERFLOW here we use log probability instead.
        self.log_prior_probs = np.log(self.prior_probs)
        self.log_likelihood_probs = np.log(self.likelihood_probs)
#         np.dstack([np.log(1 - self.likelihood_probs), np.log(self.likelihood_probs)])
        
    def predict(self, x):
        """
        Predict whether email $x is a spam or not.
        Returns:
            * y: a boolean value indicating whether $x is a spam or not.
        """        
        p_x_spam = 0
        p_x_non = 0
#         print(self.log_likelihood_probs)
        
        for word in self.wordlist:
            index = self.wordlist.index(word)
            if word in x:
                p_x_spam += self.log_likelihood_probs[1][index]
                p_x_non += self.log_likelihood_probs[0][index]
            else:
                try:
                    p_x_spam += math.log((1-self.likelihood_probs[1][index]))
                    p_x_non += math.log((1-self.likelihood_probs[0][index]))
                except:
                    print("math error, the prob is ",self.likelihood_probs[1][index], "and",(self.likelihood_probs[0][index]))
#                     print("p_x_spam is ",)
                    break
        
        p_x_spam += self.log_prior_probs[1]
        p_x_non += self.log_prior_probs[0]
        if p_x_spam >= p_x_non:
            return True
        else:
            return False

In [3]:
def read_data(filename):
    """
    Read the dataset from the file given by name $filename.
    The returned object should be a list of pairs of data. In each pair: the first element is 1 (for spam emails) 
    or 0 (for non-spam emails), the second element is a list of words in the email.
    The returned list: 
        [
            (1 , ['a', 'b', 'c']),
            (0, ['d', 'e', 'f']),
            ...
        ]
    """
    
    result = []
    
    df = pd.read_csv(filename,sep='\n',header=None,names=["email"])
    df = pd.DataFrame(df.email.str.split(' ',1).tolist(),
                                 columns = ['label','text']) 
    for row in df.itertuples(index=False):
        label = int(row.label)
        text = row.text.split(' ')
        new_data = (label,text)
        result.append(new_data)
        
    return result

    
    
def split_train(original_train_data, size=4000):
    return original_train_data[:size], original_train_data[size:]


def create_wordlist(original_train_data, threshold=26):
    """
    Create a word list from the original training set.
    Only get a word if it appears in at least $threshold emails.
    Returns:
        * a python list containing all the words that occur in at least $threshold emails.
    """
    dic = {}
    
    for i in range(len(original_train_data)):
        texts = original_train_data[i][1]
#         print(texts)
        texts = set(texts)
        for word in texts:
            if word in dic:
                dic[word]+=1
            else:
                dic[word] =1
        
    wordlist = []
    for i in dic.keys():
        if dic[i] >= threshold:
            wordlist.append(i)
    
    return wordlist

In [4]:
# threshold to determine whether to include a word in the dictionary/wordlist.
# ie. only words with frequency higher than threshold are included
THRESHOLD = 26

In [5]:
original_train_data = read_data('spam_train.txt')


# further split the data into a training set and a validation set
train_data, val_data = split_train(original_train_data)
print("test")
# Create the word list.
wordlist = create_wordlist(original_train_data, 26)
print("Total # of words:", len(wordlist))

# fit the model using train_data
model = Model(wordlist)
model.fit(train_data)
# print("shape of log: ",model.log_likelihood_probs.shape)

# TODO
# calculate the error rate on val_data (when threshold=26)
# print out the error rate
result = {"spam":0,"non-spam":0}
error_count =0
for i in range(len(val_data)):
    is_spam = model.predict(val_data[i][1])
    if is_spam == True:
        if val_data[i][0] == 0:
            error_count  += 1
    else:
        if val_data[i][0] == 1:
            error_count  += 1

error_percentage = (error_count / len(val_data))*100
print("Validation error, # = {:>4d}, % = {:>8.4f}%.".format(error_count, error_percentage))

test
Total # of words: 3048
number of spam email  1255.0
shape of data:  (4000, 2)
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.6687

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

...

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495986 -4.22938523 ... -4.58606018 -5.08505134
  -5.72104011]
 [-0.42962087 -0.66878448 -3.73528583 ... -6.03787092 -6.44333603
  -4.14075094]]
[[-0.31486535 -2.62495

In [ ]:
1-math.log(381/1256)

## 3.2 try different thresholds, find the optimal threshold (which gives minimum validation error), print out the test error at the optimal threshold

In [ ]:
def compute_error_rate(model, data):
    
    error_count = sum([y != model.predict(x) for y, x in data])
    return 100.0 * error_count / len(data)

In [ ]:
thresholds = list(range(1, 35))
train_error = []
val_error = []
test_error = []
original_train_data = read_data('spam_train.txt')
train_data, val_data = split_train(original_train_data)
test_data = read_data('spam_test.txt')

In [ ]:
for th in thresholds[::-1]:
    print('With threshold {}....'.format(th))
    time1 = time.time()

    # vocabulary selection
    wordlist = create_wordlist(original_train_data, th)

    # fit model using the wordlist
    model = Model(wordlist)
    model.fit(train_data)
    
    print("finish training data ...")
    # compute classification error rates
    err_train = compute_error_rate(model, train_data)
    err_val = compute_error_rate(model, val_data)
    err_test = compute_error_rate(model, test_data)

    # store results for plotting
    train_error.append(err_train)
    val_error.append(err_val)
    test_error.append(err_test)

    time2 = time.time()
    print("train:{} val:{} test:{} len(V)={}".format(err_train, err_val, err_test, len(wordlist)))
    print('time: {}'.format(time2 - time1))

In [ ]:
# plot the training and validation error rate vs. the thresholds
# choose the threshold with the minimal validation error rate and report the corresponding test error rate

# TODO
plt.subplot(2, 1, 1)
plt.plot(thresholds,train_error[::-1])
plt.xlabel("threshold")
plt.ylabel("train_error")
plt.title("train_error vs threshold")
plt.subplot(2, 1, 2)
plt.plot(thresholds,val_error[::-1],label="val error vs. threshold")
plt.xlabel("threshold")
plt.ylabel("val_error")
plt.title("val_error vs threshold")
plt.legend()
plt.show()

opt = thresholds[val_error.index(min(val_error))]

print('Best performance at validated threshold {} with test error rate {}.'.format(opt, test_error[opt]))